In [28]:
ROOT_DIR = "data"

In [2]:
!pip install miceforest

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 12.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 27.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

In [30]:
train_data = pd.read_csv("train.csv")

In [31]:
train_data.shape

(40506, 464)

In [32]:
train_data.isna().sum()

Wip Line_Dam                         0
Process Desc._Dam                    0
Equipment_Dam                        0
Model.Suffix_Dam                     0
Workorder_Dam                        0
                                 ...  
Receip No Judge Value_Fill2      40506
WorkMode Collect Result_Fill2        0
WorkMode Unit Time_Fill2         40506
WorkMode Judge Value_Fill2       40506
target                               0
Length: 464, dtype: int64

In [33]:
cols = train_data.columns
null_list = []

for i in cols:
    if train_data[i].isna().sum() == train_data.shape[0]:
        null_list.append(i)

print(null_list)

['CURE END POSITION X Unit Time_Dam', 'CURE END POSITION X Judge Value_Dam', 'CURE END POSITION Z Unit Time_Dam', 'CURE END POSITION Z Judge Value_Dam', 'CURE END POSITION Θ Unit Time_Dam', 'CURE END POSITION Θ Judge Value_Dam', 'CURE SPEED Unit Time_Dam', 'CURE SPEED Judge Value_Dam', 'CURE STANDBY POSITION X Unit Time_Dam', 'CURE STANDBY POSITION X Judge Value_Dam', 'CURE STANDBY POSITION Z Unit Time_Dam', 'CURE STANDBY POSITION Z Judge Value_Dam', 'CURE STANDBY POSITION Θ Unit Time_Dam', 'CURE STANDBY POSITION Θ Judge Value_Dam', 'CURE START POSITION X Unit Time_Dam', 'CURE START POSITION X Judge Value_Dam', 'CURE START POSITION Z Unit Time_Dam', 'CURE START POSITION Z Judge Value_Dam', 'CURE START POSITION Θ Unit Time_Dam', 'CURE START POSITION Θ Judge Value_Dam', 'DISCHARGED SPEED OF RESIN Unit Time_Dam', 'DISCHARGED SPEED OF RESIN Judge Value_Dam', 'DISCHARGED TIME OF RESIN(Stage1) Unit Time_Dam', 'DISCHARGED TIME OF RESIN(Stage1) Judge Value_Dam', 'DISCHARGED TIME OF RESIN(Stage

In [34]:
train_data['DISCHARGED TIME OF RESIN(Stage3) Unit Time_Dam']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
40501   NaN
40502   NaN
40503   NaN
40504   NaN
40505   NaN
Name: DISCHARGED TIME OF RESIN(Stage3) Unit Time_Dam, Length: 40506, dtype: float64

In [35]:
train_data['DISCHARGED TIME OF RESIN(Stage3) Unit Time_Dam'].unique()

array([nan])

In [36]:
print(len(null_list))

278


제외할 데이터
1. 모두 NULL
2. 모두 같은 데이터

In [37]:
# 모두 NULL 인 Columns drop
train_not_null = train_data.drop(columns = null_list)

In [38]:
train_not_null.shape

(40506, 186)

In [39]:
cols2 = train_not_null.columns
unique_cols = []

for i in cols2:
    if train_not_null[i].nunique() == 1:
        unique_cols.append(i)
        
print(unique_cols)

['Wip Line_Dam', 'Process Desc._Dam', 'Insp. Seq No._Dam', 'Insp Judge Code_Dam', 'CURE STANDBY POSITION X Collect Result_Dam', 'CURE STANDBY POSITION Z Collect Result_Dam', 'CURE STANDBY POSITION Θ Collect Result_Dam', 'CURE START POSITION Z Collect Result_Dam', 'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam', 'Wip Line_AutoClave', 'Process Desc._AutoClave', 'Equipment_AutoClave', 'Insp. Seq No._AutoClave', 'Insp Judge Code_AutoClave', '1st Pressure Judge Value_AutoClave', '2nd Pressure Judge Value_AutoClave', '3rd Pressure Judge Value_AutoClave', 'GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave', 'GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave', 'Wip Line_Fill1', 'Process Desc._Fill1', 'Insp. Seq No._Fill1', 'Insp Judge Code_Fill1', 'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1', 'Wip Line_Fill2', 'Process Desc._Fill2', 'Insp. Seq No._Fill2', 'Insp Judge Code_Fill2', 'CURE END POSITION Θ Collect Result_Fill2', 'CURE STANDBY POSITION X Collect Result_Fill2',

In [40]:
train_data = train_not_null.drop(columns = unique_cols)

In [41]:
train_data.shape

(40506, 146)

In [42]:
df = train_data

In [43]:
def detect_column_types(df):
    col_types = {}
    for col in df.columns:
        unique_types = df[col].apply(type).unique()  # 각 칼럼의 데이터 타입 확인
        col_types[col] = unique_types
    return col_types

In [44]:
column_types = detect_column_types(df)

In [45]:
float_list = []
int_list = []
other_list = []

for col, types in column_types.items():
    if float in types:
        float_list.append(col)
    elif int in types:
        int_list.append(col)
    else:
        other_list.append(col)

In [46]:
column_types2 = detect_column_types(df[other_list])

In [47]:
for col, types in column_types2.items():
    print(f"칼럼 '{col}'의 데이터 타입들:")
    for t in types:
        print(f" - {t}")
    print("\n")

칼럼 'Equipment_Dam'의 데이터 타입들:
 - <class 'str'>


칼럼 'Model.Suffix_Dam'의 데이터 타입들:
 - <class 'str'>


칼럼 'Workorder_Dam'의 데이터 타입들:
 - <class 'str'>


칼럼 'Model.Suffix_AutoClave'의 데이터 타입들:
 - <class 'str'>


칼럼 'Workorder_AutoClave'의 데이터 타입들:
 - <class 'str'>


칼럼 'Chamber Temp. Judge Value_AutoClave'의 데이터 타입들:
 - <class 'str'>


칼럼 'Equipment_Fill1'의 데이터 타입들:
 - <class 'str'>


칼럼 'Model.Suffix_Fill1'의 데이터 타입들:
 - <class 'str'>


칼럼 'Workorder_Fill1'의 데이터 타입들:
 - <class 'str'>


칼럼 'Equipment_Fill2'의 데이터 타입들:
 - <class 'str'>


칼럼 'Model.Suffix_Fill2'의 데이터 타입들:
 - <class 'str'>


칼럼 'Workorder_Fill2'의 데이터 타입들:
 - <class 'str'>


칼럼 'target'의 데이터 타입들:
 - <class 'str'>




In [48]:
column_types2

{'Equipment_Dam': array([<class 'str'>], dtype=object),
 'Model.Suffix_Dam': array([<class 'str'>], dtype=object),
 'Workorder_Dam': array([<class 'str'>], dtype=object),
 'Model.Suffix_AutoClave': array([<class 'str'>], dtype=object),
 'Workorder_AutoClave': array([<class 'str'>], dtype=object),
 'Chamber Temp. Judge Value_AutoClave': array([<class 'str'>], dtype=object),
 'Equipment_Fill1': array([<class 'str'>], dtype=object),
 'Model.Suffix_Fill1': array([<class 'str'>], dtype=object),
 'Workorder_Fill1': array([<class 'str'>], dtype=object),
 'Equipment_Fill2': array([<class 'str'>], dtype=object),
 'Model.Suffix_Fill2': array([<class 'str'>], dtype=object),
 'Workorder_Fill2': array([<class 'str'>], dtype=object),
 'target': array([<class 'str'>], dtype=object)}

In [49]:
a= train_data.columns
null_list = a[train_data.isnull().sum() > 0]
null_list

Index(['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
       'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
       'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'],
      dtype='object')

In [50]:
OK_data1 = "HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam"
OK_data2 = 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'
OK_data3 = 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'

In [51]:
train_data[OK_data1].value_counts()

HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam
OK       11293
549       7271
162.4     3580
550       2398
550.3     1909
549.5     1263
548.5       26
Name: count, dtype: int64

In [53]:
train_data[OK_data1].replace('OK', np.nan, inplace=True)
train_data[OK_data2].replace('OK', np.nan, inplace=True)
train_data[OK_data3].replace('OK', np.nan, inplace=True)

In [55]:
train_data[OK_data1].value_counts()

HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam
549      7271
162.4    3580
550      2398
550.3    1909
549.5    1263
548.5      26
Name: count, dtype: int64

In [56]:
train_data[OK_data1].value_counts()

HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam
549      7271
162.4    3580
550      2398
550.3    1909
549.5    1263
548.5      26
Name: count, dtype: int64

In [44]:
column_types2

{'Equipment_Dam': array([<class 'str'>], dtype=object),
 'Model.Suffix_Dam': array([<class 'str'>], dtype=object),
 'Workorder_Dam': array([<class 'str'>], dtype=object),
 'Model.Suffix_AutoClave': array([<class 'str'>], dtype=object),
 'Workorder_AutoClave': array([<class 'str'>], dtype=object),
 'Chamber Temp. Judge Value_AutoClave': array([<class 'str'>], dtype=object),
 'Equipment_Fill1': array([<class 'str'>], dtype=object),
 'Model.Suffix_Fill1': array([<class 'str'>], dtype=object),
 'Workorder_Fill1': array([<class 'str'>], dtype=object),
 'Equipment_Fill2': array([<class 'str'>], dtype=object),
 'Model.Suffix_Fill2': array([<class 'str'>], dtype=object),
 'Workorder_Fill2': array([<class 'str'>], dtype=object),
 'target': array([<class 'str'>], dtype=object)}

In [45]:
object_columns = df.select_dtypes(include='object').columns

In [46]:
object_columns

Index(['Equipment_Dam', 'Model.Suffix_Dam', 'Workorder_Dam',
       'Model.Suffix_AutoClave', 'Workorder_AutoClave',
       'Chamber Temp. Judge Value_AutoClave', 'Equipment_Fill1',
       'Model.Suffix_Fill1', 'Workorder_Fill1', 'Equipment_Fill2',
       'Model.Suffix_Fill2', 'Workorder_Fill2', 'target'],
      dtype='object')

In [47]:
print(train_data[object_columns[0]].value_counts())

Equipment_Dam
Dam dispenser #1    25030
Dam dispenser #2    15476
Name: count, dtype: int64


In [48]:
train_data[object_columns[0]] = np.where(train_data[object_columns[0]]=='Dam dispenser #1', 1, 2)

In [49]:
train_data[object_columns[0]]

0        1
1        1
2        2
3        2
4        1
        ..
40501    1
40502    2
40503    1
40504    2
40505    1
Name: Equipment_Dam, Length: 40506, dtype: int64

In [50]:
train_data[object_columns[1]].value_counts()

Model.Suffix_Dam
AJX75334501    33820
AJX75334502     3390
AJX75334505     2635
AJX75334507      310
AJX75334503      162
AJX75334506      129
AJX75334508       60
Name: count, dtype: int64

In [51]:
def process_head_normal1(data):
    if data == "AJX75334501":
        return 1
    elif data == "AJX75334502":
        return 2
    elif data == "AJX75334503":
        return 3
    elif data == "AJX75334505":
        return 5
    elif data == "AJX75334506":
        return 6
    elif data == "AJX75334507":
        return 7
    elif data == "AJX75334508":
        return 8

In [52]:
train_data[object_columns[1]]

0        AJX75334505
1        AJX75334505
2        AJX75334501
3        AJX75334501
4        AJX75334501
            ...     
40501    AJX75334501
40502    AJX75334501
40503    AJX75334501
40504    AJX75334501
40505    AJX75334501
Name: Model.Suffix_Dam, Length: 40506, dtype: object

In [53]:
train_data[object_columns[1]] = train_data[object_columns[1]].apply(process_head_normal1)

In [54]:
train_data[object_columns[1]]

0        5
1        5
2        1
3        1
4        1
        ..
40501    1
40502    1
40503    1
40504    1
40505    1
Name: Model.Suffix_Dam, Length: 40506, dtype: int64

In [55]:
train_data[object_columns[2]].value_counts()

Workorder_Dam
3L1X8574-2       272
3K1XC268-1       227
3L1X9934-1       217
3L1X8506-1       217
3K1XA586-2       189
                ... 
4BPM0084-1         2
3FPXX064-0003      2
3H1XB714-1         1
4CPM0166-1         1
3HPM0061-1         1
Name: count, Length: 663, dtype: int64

In [56]:
value_counts2 = train_data[object_columns[2]].value_counts()

In [57]:
mapping3 = {value: str(index + 1) for index, value in enumerate(value_counts2.index)}

In [58]:
train_data[object_columns[2]] = train_data[object_columns[2]].map(mapping3)

In [59]:
train_data[object_columns[2]].value_counts()

Workorder_Dam
1      272
2      227
3      217
4      217
5      189
      ... 
659      2
660      2
661      1
662      1
663      1
Name: count, Length: 663, dtype: int64

In [60]:
train_data[object_columns[3]].value_counts()

Model.Suffix_AutoClave
AJX75334501    33820
AJX75334502     3390
AJX75334505     2635
AJX75334507      310
AJX75334503      162
AJX75334506      129
AJX75334508       60
Name: count, dtype: int64

In [61]:
train_data[object_columns[3]] = train_data[object_columns[3]].apply(process_head_normal1)

In [62]:
train_data[object_columns[3]].value_counts()

Model.Suffix_AutoClave
1    33820
2     3390
5     2635
7      310
3      162
6      129
8       60
Name: count, dtype: int64

In [63]:
train_data[object_columns[4]].value_counts()

Workorder_AutoClave
3L1X8574-2       272
3K1XC268-1       227
3L1X9934-1       217
3L1X8506-1       217
3K1XA586-2       189
                ... 
4BPM0084-1         2
3FPXX064-0003      2
3H1XB714-1         1
4CPM0166-1         1
3HPM0061-1         1
Name: count, Length: 663, dtype: int64

In [64]:
value_counts4 = train_data[object_columns[4]].value_counts()
mapping4 = {value: str(index + 1) for index, value in enumerate(value_counts4.index)}
train_data[object_columns[4]] = train_data[object_columns[4]].map(mapping4)

In [65]:
train_data[object_columns[4]].value_counts()

Workorder_AutoClave
1      272
2      227
3      217
4      217
5      189
      ... 
659      2
660      2
661      1
662      1
663      1
Name: count, Length: 663, dtype: int64

In [66]:
train_data[object_columns[5]].value_counts()

Chamber Temp. Judge Value_AutoClave
OK    29112
NG    11394
Name: count, dtype: int64

In [67]:
train_data[object_columns[5]] = np.where(train_data[object_columns[5]]=='OK', 0, 1)

In [68]:
train_data[object_columns[5]].value_counts()

Chamber Temp. Judge Value_AutoClave
0    29112
1    11394
Name: count, dtype: int64

In [69]:
train_data[object_columns[6]].value_counts()

Equipment_Fill1
Fill1 dispenser #1    25027
Fill1 dispenser #2    15479
Name: count, dtype: int64

In [70]:
train_data[object_columns[6]] = np.where(train_data[object_columns[6]]=='Fill1 dispenser #1', 1, 2)

In [71]:
train_data[object_columns[6]].value_counts()

Equipment_Fill1
1    25027
2    15479
Name: count, dtype: int64

In [72]:
train_data[object_columns[6]].value_counts()

Equipment_Fill1
1    25027
2    15479
Name: count, dtype: int64

In [73]:
train_data[object_columns[7]].value_counts()

Model.Suffix_Fill1
AJX75334501    33820
AJX75334502     3390
AJX75334505     2635
AJX75334507      310
AJX75334503      162
AJX75334506      129
AJX75334508       60
Name: count, dtype: int64

In [74]:
train_data[object_columns[7]] = train_data[object_columns[7]].apply(process_head_normal1)

In [75]:
train_data[object_columns[7]].value_counts()

Model.Suffix_Fill1
1    33820
2     3390
5     2635
7      310
3      162
6      129
8       60
Name: count, dtype: int64

In [76]:
train_data[object_columns[8]].value_counts()

Workorder_Fill1
3L1X8574-2       272
3K1XC268-1       227
3L1X9934-1       217
3L1X8506-1       217
3K1XA586-2       189
                ... 
4BPM0084-1         2
3FPXX064-0003      2
3H1XB714-1         1
4CPM0166-1         1
3HPM0061-1         1
Name: count, Length: 663, dtype: int64

In [77]:
value_counts8 = train_data[object_columns[8]].value_counts()
mapping8 = {value: str(index + 1) for index, value in enumerate(value_counts8.index)}
train_data[object_columns[8]] = train_data[object_columns[8]].map(mapping8)

In [78]:
train_data[object_columns[8]].value_counts()

Workorder_Fill1
1      272
2      227
3      217
4      217
5      189
      ... 
659      2
660      2
661      1
662      1
663      1
Name: count, Length: 663, dtype: int64

In [79]:
train_data[object_columns[9]].value_counts()

Equipment_Fill2
Fill2 dispenser #1    25026
Fill2 dispenser #2    15480
Name: count, dtype: int64

In [80]:
train_data[object_columns[9]] = np.where(train_data[object_columns[9]]=='Fill2 dispenser #1', 1, 2)

In [81]:
train_data[object_columns[9]].value_counts()

Equipment_Fill2
1    25026
2    15480
Name: count, dtype: int64

In [82]:
train_data[object_columns[10]].value_counts()

Model.Suffix_Fill2
AJX75334501    33820
AJX75334502     3390
AJX75334505     2635
AJX75334507      310
AJX75334503      162
AJX75334506      129
AJX75334508       60
Name: count, dtype: int64

In [83]:
value_counts10 = train_data[object_columns[10]].value_counts()
mapping10 = {value: str(index + 1) for index, value in enumerate(value_counts10.index)}
train_data[object_columns[10]] = train_data[object_columns[10]].map(mapping10)

In [84]:
train_data[object_columns[10]].value_counts()

Model.Suffix_Fill2
1    33820
2     3390
3     2635
4      310
5      162
6      129
7       60
Name: count, dtype: int64

In [85]:
train_data[object_columns[11]].value_counts()

Workorder_Fill2
3L1X8574-2       272
3K1XC268-1       227
3L1X9934-1       217
3L1X8506-1       217
3K1XA586-2       189
                ... 
4BPM0084-1         2
3FPXX064-0003      2
3H1XB714-1         1
4CPM0166-1         1
3HPM0061-1         1
Name: count, Length: 663, dtype: int64

In [86]:
value_counts11 = train_data[object_columns[11]].value_counts()
mapping11 = {value: str(index + 1) for index, value in enumerate(value_counts11.index)}
train_data[object_columns[11]] = train_data[object_columns[11]].map(mapping11)

In [87]:
train_data[object_columns[11]].value_counts()

Workorder_Fill2
1      272
2      227
3      217
4      217
5      189
      ... 
659      2
660      2
661      1
662      1
663      1
Name: count, Length: 663, dtype: int64

In [88]:
pip install imblearn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [89]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 데이터프레임 복사
df = train_data.copy()

# Target 열을 Label Encoding: Normal -> 0, Abnormal -> 1
label_encoder = LabelEncoder()
df['target'] = label_encoder.fit_transform(df['target'])

# X, y로 데이터셋 분리
X = df.drop('target', axis=1)
y = df['target']

# 불균형 데이터 조정 (언더샘플링)
rus = RandomUnderSampler(random_state=110)
X_resampled, y_resampled = rus.fit_resample(X, y)

# 범주형 및 수치형 열 분리
categorical_cols = X_resampled.select_dtypes(include=['category']).columns
numerical_cols = X_resampled.select_dtypes(include=['float64', 'int64']).columns

# OneHotEncoder를 사용하여 범주형 데이터 변환
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(drop='first'), categorical_cols)])

# 학습용 및 테스트용 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=110, stratify=y_resampled)

# XGBoost 모델 정의 및 학습을 위한 파이프라인 생성
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss',
        eta=0.1,
        max_depth=6,
    
    ))
])

# 모델 학습
model.fit(X_train, y_train)

# 예측 수행
y_pred = model.predict(X_test)

# 모델 평가
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))



Accuracy: 0.5893617021276596
Confusion Matrix:
 [[272 198]
 [188 282]]
Classification Report:
               precision    recall  f1-score   support

           0       0.59      0.58      0.58       470
           1       0.59      0.60      0.59       470

    accuracy                           0.59       940
   macro avg       0.59      0.59      0.59       940
weighted avg       0.59      0.59      0.59       940



In [95]:
test_data = pd.read_csv("test.csv")

In [98]:
test_data['Equipment_Dam'] = np.where(test_data['Equipment_Dam']=='Dam dispenser #1', 1, 2)

In [100]:
column_types2

{'Equipment_Dam': array([<class 'str'>], dtype=object),
 'Model.Suffix_Dam': array([<class 'str'>], dtype=object),
 'Workorder_Dam': array([<class 'str'>], dtype=object),
 'Model.Suffix_AutoClave': array([<class 'str'>], dtype=object),
 'Workorder_AutoClave': array([<class 'str'>], dtype=object),
 'Chamber Temp. Judge Value_AutoClave': array([<class 'str'>], dtype=object),
 'Equipment_Fill1': array([<class 'str'>], dtype=object),
 'Model.Suffix_Fill1': array([<class 'str'>], dtype=object),
 'Workorder_Fill1': array([<class 'str'>], dtype=object),
 'Equipment_Fill2': array([<class 'str'>], dtype=object),
 'Model.Suffix_Fill2': array([<class 'str'>], dtype=object),
 'Workorder_Fill2': array([<class 'str'>], dtype=object),
 'target': array([<class 'str'>], dtype=object)}

In [101]:
def process_head_normal1(data):
    if data == "AJX75334501":
        return 1
    elif data == "AJX75334502":
        return 2
    elif data == "AJX75334503":
        return 3
    elif data == "AJX75334505":
        return 5
    elif data == "AJX75334506":
        return 6
    elif data == "AJX75334507":
        return 7
    elif data == "AJX75334508":
        return 8

test_data[object_columns[1]] = test_data[object_columns[1]].apply(process_head_normal1)

value_counts2 = test_data[object_columns[2]].value_counts()
mapping3 = {value: str(index + 1) for index, value in enumerate(value_counts2.index)}
test_data[object_columns[2]] = test_data[object_columns[2]].map(mapping3)

test_data[object_columns[3]] = test_data[object_columns[3]].apply(process_head_normal1)

value_counts4 = test_data[object_columns[4]].value_counts()
mapping4 = {value: str(index + 1) for index, value in enumerate(value_counts4.index)}
test_data[object_columns[4]] = test_data[object_columns[4]].map(mapping4)

test_data[object_columns[5]] = np.where(test_data[object_columns[5]]=='OK', 0, 1)

test_data[object_columns[6]] = np.where(test_data[object_columns[6]]=='Fill1 dispenser #1', 1, 2)

test_data[object_columns[7]] = test_data[object_columns[7]].apply(process_head_normal1)

value_counts8 = test_data[object_columns[8]].value_counts()
mapping8 = {value: str(index + 1) for index, value in enumerate(value_counts8.index)}
test_data[object_columns[8]] = test_data[object_columns[8]].map(mapping8)

test_data[object_columns[9]] = np.where(test_data[object_columns[9]]=='Fill2 dispenser #1', 1, 2)

value_counts10 = test_data[object_columns[10]].value_counts()
mapping10 = {value: str(index + 1) for index, value in enumerate(value_counts10.index)}
test_data[object_columns[10]] = test_data[object_columns[10]].map(mapping10)

value_counts11 = test_data[object_columns[11]].value_counts()
mapping11 = {value: str(index + 1) for index, value in enumerate(value_counts11.index)}
test_data[object_columns[11]] = test_data[object_columns[11]].map(mapping11)

In [103]:
test_data[OK_data1] = np.where(test_data[OK_data1]=='OK', 0, 1)
test_data[OK_data2] = np.where(test_data[OK_data2]=='OK', 0, 1)
test_data[OK_data3] = np.where(test_data[OK_data3]=='OK', 0, 1)

In [109]:
# 테스트 데이터 로드 및 전처리

df_test_x = test_data[X.columns]  # 학습 데이터의 특징들과 동일한 열 선택

# 테스트 데이터에 대한 예측 수행
test_pred = model.predict(df_test_x)

# 제출 파일 생성
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = test_pred
df_sub['target'] = df_sub['target'].map({1: 'Normal', 0: 'AbNormal'})
df_sub.to_csv("submission.csv", index=False)


In [110]:
df_sub

,Set ID,target
0,0001be084fbc4aaa9d921f39e595961b,AbNormal
1,0005bbd180064abd99e63f9ed3e1ac80,Normal
2,000948934c4140d883d670adcb609584,AbNormal
3,000a6bfd02874c6296dc7b2e9c5678a7,Normal
4,0018e78ce91343678716e2ea27a51c95,Normal
...,...,...
17356,ffea508b59934d689b540f95eb3fa730,Normal
17357,ffed8923c8a448a98afc641b770be153,Normal
17358,fff1e73734da40adbe805359b3efb462,Normal
17359,fff8e38bdd09470baf95f71e92075dec,Normal


In [111]:
df_sub['target'].value_counts()

target
Normal      10634
AbNormal     6727
Name: count, dtype: int64

0.15~